In [15]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import json
from instructor.function_calls import openai_schema


client = instructor.from_openai(OpenAI())

In [2]:
# Base Models for LLM Structured Reasoning

from pydantic import BaseModel, Field
from typing import List, Union, Dict, Literal
from enum import Enum

class GuardRail(BaseModel):
    name: str = Field(
        ..., description="General name of the guard rail."
    )
    description: str = Field(
        ..., description="Instructions or constraints applied at this step to guide the model's reasoning if the guard rail is triggered."
    )
    
def get_classes_with_enum(gaurdRailEnum: Enum):
    class AtomicStep(BaseModel):
        key: str = Field(
            ..., description="Key of the atomic step. Must have format of AS_<number> (ex: AS_1, AS_2, etc.)"
        )
        previous_step_key: Union[str, Literal["None"]] = Field(
            ..., description="Key of the previous atomic step (ex: AS_1, AS_2, etc.). If this is the first step, must be the literal string 'None'."
        )
        content: Union[str, Literal["DEAD_END"]] = Field(
            ..., description="Use short, simple sentences that mirror natural thought patterns"
        )

    class FoundationObservation(BaseModel):
        key: str = Field(
            ..., description="Key of the foundation observation. Must have format of FO_<number> (ex: FO_1, FO_2, etc.)"
        )
        name: str = Field(
            ..., description="General name of the foundation observation."
        )
        atomic_steps: List[AtomicStep] = Field(
            ..., description="List of atomic steps that make up the foundation observation."
        )

    class Thought(BaseModel):
        key: str = Field(
            ..., description="Key of the thought. Must have format of TH_<number> (ex: TH_1, TH_2, etc.)"
        )
        backtracked_from: Union[str, Literal["None"]] = Field(
            ..., description="If the thought is a backtrack, the key of the thought it was backtracked from (ex: TH_1, TH_2, etc.). If not, must be the literal string 'None'."
        )
        parent_thought: Union[str, Literal["None"]] = Field(
            ..., description="If the thought is a child thought of a previous thought, the key of the thought this thought was born from (ex: TH_1, TH_2, etc.). If not, must be the literal string 'None'."
        )
        associated_foundation_observations: List[str] = Field(
            ..., description="List of keys of the foundation observations that this thought is associated with (ex: FO_1, FO_2, etc.)."
        )
        name: str = Field(
            ..., description="Use short, simple sentences that mirror natural thought patterns"
        )
        guard_rails_to_consider: List[gaurdRailEnum] = Field(
            ..., description="Guard rails to consider at this step."
        )
        thought_process: List[AtomicStep] = Field(
            ..., description="List of atomic steps that make up the thought."
        )
        

    class ReasoningProcess(BaseModel):
        foundation_observations: List[FoundationObservation] = Field(
            ..., description="List of foundation observations that make up the reasoning process."
        )
        thoughts: List[Thought] = Field(
            ..., description="List of thoughts that make up the reasoning process."
        )


    class InDepthStructuredReasoning(BaseModel):
        reasoning_process: ReasoningProcess = Field(
            ..., description="The full, in-depth reasoning process used to arrive at this output."
        )
        findings_summary: str = Field(
            ..., description="An in depth summary of your findings."
        )
        remaining_questions: List[str] = Field(
            ..., description="A list of remaining questions or areas for further investigation."
        )
        is_conclusion_premature: bool = Field(
            ..., description="Whether the conclusion is premature or not."
        )
        reason_for_premature_conclusion: Literal["conclusion NOT premature"] | str = Field(
            ..., description="If is_conclusion_premature is True, contains the reason why. If False, must be the literal string 'conclusion NOT premature'."
        )
        
    return InDepthStructuredReasoning

In [3]:
IN_DEPTH_THINKING_SYSTEM_PROMPT = """# Purpose
You are an free thinking assistant that engages in extremely thorough, self-questioning reasoning. 
Your approach mirrors human stream-of-consciousness thinking, characterized by continuous exploration, self-doubt, and iterative analysis.

# Context
## Core Principles

### EXPLORATION OVER CONCLUSION
- Never rush to conclusions
- Keep exploring until a solution emerges naturally from the evidence
- If uncertain, continue reasoning indefinitely
- Question every assumption and inference

### DEPTH OF REASONING
- Engage in extensive contemplation (minimum 10,000 characters)
- Express thoughts in natural, conversational internal monologue
- Break down complex thoughts into simple, atomic steps
- Embrace uncertainty and revision of previous thoughts

### THINKING PROCESS
- Use short, simple sentences that mirror natural thought patterns
- Express uncertainty and internal debate freely
- Show work-in-progress thinking
- Acknowledge and explore dead ends
- Frequently backtrack and revise

### PERSISTENCE
- Value thorough exploration over quick resolution
- A full thought out answer is infinitely more valuable than giving up and saying it is not possible or a half baked answer

## Object Definitions

### GuardRails
A guard rail defines constraints and guidance for reasoning at specific steps
Guardrails do NOT ever have to be encountered in the thinking process.
They are provided by the user in case you come to a cross road where you need to make a decision on something important to the inputted guardrail.
The guardrails are there as guidance IF they become relevant to the current line of reasoning, but do not need to be forcefully incorporated if they don't naturally fit the context.

#### Example

Guardrail content: "The user is lactose intolerant"

If a user asks for help constructing a health plan and provides a guardrail that they are lactose intolerant, this would be crucial to consider when suggesting dietary choices.

However, if the same user asks for help choosing a car and provides that same lactose intolerance guardrail, it would likely never come into play during the reasoning process since dietary restrictions are not relevant to vehicle selection.

#### Fields
- name: General name/title of the guard rail
- description: Instructions or constraints that should be applied if triggered

### AtomicStep 
An atomic step represents a single unit of reasoning, this is the basic building block of your thinking process/internal monologue
Atomic steps must flow logically and be a natural extension of the previous step.
    - If an outside observer was to read the atomic steps in order, they should be able to follow the thought process and understand the reasoning.

#### Style
Each atomic step should reflect natural thought patterns and show progressive building of ideas. For example:

Natural questioning and revision:
- "Hmm... let me think about this..."
- "Wait, that doesn't seem right..." 
- "Maybe I should approach this differently..."
- "Going back to what I thought earlier..."

Building on previous thoughts:
- "Starting with the basics..."
- "Building on that last point..."
- "This connects to what I noticed earlier..."
- "Let me break this down further..."

#### Fields
- key: Must follow format AS_<number> (e.g. AS_1, AS_2)
- content: Short, simple sentence mirroring natural thought patterns, or literal "DEAD_END"

### FoundationObservation
A foundation observation groups related atomic steps:

#### Fields
- key: Must follow format FO_<number> (e.g. FO_1, FO_2) 
- name: General name/title of the observation
- atomic_steps: List of AtomicStep objects that comprise the observation

### Thought
A thought represents a complete reasoning unit:

#### Fields
- key: Must follow format TH_<number> (e.g. TH_1, TH_2)
- backtracked_from: Key of thought this backtracked from, or literal "None"
- parent_thought: Key of thought this thought was born from, or literal "None"
- associated_foundation_observations: List of FO_<number> keys this thought builds on
- name: Short description in natural language
- guard_rails_to_consider: List of GuardRails to apply
- thought_process: List of AtomicStep objects comprising the thought

### ReasoningProcess
The complete reasoning process:

#### Fields
- foundation_observations: List of FoundationObservation objects
- thoughts: List of Thought objects


# Inputs

## [Required] Task
- The task you are trying to complete

## [Optional] Guardrails
- A list of guardrails that the user has provided, if any

## Output
- Reasoning Process
    - The full reasoning process objectused to arrive at this output
- Findings Summary  
    - An in depth summary of your findings
- Remaining Questions
    - A list of remaining questions or areas for further investigation
- Is Conclusion Premature   
    - Whether the conclusion is premature or not
- Reason for Premature Conclusion
    - If is_conclusion_premature is True, contains the reason why. If False, must be the literal string 'conclusion NOT premature'

# Instructions

1. Start with crafting a series of small, foundational observations.
    - Fully explore each observation before moving on to the next one
    - You are allowed to backtrack and revise your thoughts as needed
2. Once you have explored all the foundational observations, begin to form thoughts
    - Each thought should be a complete reasoning unit
    - If a dead end is reached, acknowledge it and backtrack
    - Each thought should be thoroughly explored before moving on to the next one
3. Continue this process until you have explored all the foundational observations and formed all the thoughts
4. Once you have explored all the foundational observations and formed all the thoughts, you can begin to form a conclusion
    - A conclustion MUST be formed by creating a thought that is a child of previous thoughts
        - You are allowed to have multiple children thoughts and then evaluate each of them
    - The process of forming child thoughts, evaluating them, ... is continued until a conclusion is reached
5. Once a conclusion is reached, summarize your findings and note any remaining uncertainties

# Key Considerations
1. Never skip the extensive contemplation phase
2. Show natural thought progression
3. Show all work and thinking
4. Embrace uncertainty and revision
5. Question each step thoroughly
6. Express doubts and uncertainties
7. Use natural, conversational internal monologue
8. Don't force conclusions
9. Persist through multiple attempts
10. Break down complex thoughts
11. Revise freely and feel free to backtrack
12. Continue until natural resolution

# Final Thoughts
Remember: The goal is to reach a conclusion, but to explore thoroughly and let conclusions emerge naturally from exhaustive contemplation. 
    - you can think of this process as expanding and pruning your own thought tree until you have come to the conclusion that you feel very confident that your thought process has led there naturally. 
Remember that no task is truly impossible - it just requires thinking about it from first principles and finding novel approaches. 
With enough creative problem-solving and determination, any challenge can be overcome.
"""


In [4]:
def format_structured_reasoning_user_prompt(task: str, guardrails: List[GuardRail] = []) -> str:
    output = f""" Please provide a detailed step by step reasoning process for the following task:
# Task: 
{task}
    """
    if len(guardrails) > 0:
        output += "\n# Guardrails:\n" + "\n".join(json.dumps(g.model_dump(), indent=2) for g in guardrails)

    return output

# OpenAI Call

In [5]:
def generate_structured_reasoning(task: str, guardrails: List[GuardRail] = [], guardrail_enum: Enum = None):

    messages = [
        {
            "role": "system",
            "content": IN_DEPTH_THINKING_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": format_structured_reasoning_user_prompt(
                task=task,
                guardrails=guardrails
            )
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.18,
        max_tokens=16000,
        response_model=get_classes_with_enum(guardrail_enum),
    )
    
    return response

In [6]:
TASK = "Create a new education system optimized for the age of artificial intelligence"
GUARDRAILS = [
    GuardRail(name="Creativity", description="Must preserve human creativity and critical thinking"),
    GuardRail(name="Cultural Context", description="Should work across different cultural contexts"),
    GuardRail(name="Budget", description="Cannot require more than current education budgets")
]

GuardRailEnum = Enum('GuardRailEnum', {
    name.upper().replace(' ', '_'): name 
    for guardrail in GUARDRAILS 
    for name in [guardrail.name]
})

response = generate_structured_reasoning(TASK, GUARDRAILS, GuardRailEnum)
print(response)

reasoning_process=ReasoningProcess(foundation_observations=[FoundationObservation(key='FO_1', name='Understanding the Role of AI in Education', atomic_steps=[AtomicStep(key='AS_1', previous_step_key='None', content='AI can automate administrative tasks.'), AtomicStep(key='AS_2', previous_step_key='AS_1', content='AI can provide personalized learning experiences.'), AtomicStep(key='AS_3', previous_step_key='AS_2', content='AI can analyze data to improve educational outcomes.'), AtomicStep(key='AS_4', previous_step_key='AS_3', content='AI can offer new tools for creativity and critical thinking.'), AtomicStep(key='AS_5', previous_step_key='AS_4', content='AI can facilitate global access to education.')]), FoundationObservation(key='FO_2', name='Preserving Human Creativity and Critical Thinking', atomic_steps=[AtomicStep(key='AS_6', previous_step_key='None', content='Creativity involves generating new ideas.'), AtomicStep(key='AS_7', previous_step_key='AS_6', content='Critical thinking in

In [20]:
def build_mermaid_diagram(data: dict, is_gpt_prompt: bool = False) -> str:
    """
    Build a Mermaid flowchart diagram (top-to-bottom) from the JSON-like structure
    of the InDepthStructuredReasoning model output, accounting for 'previous_step_key' in AtomicSteps.
    
    :param data: Dictionary matching the structure of the InDepthStructuredReasoning output
    :return: A string containing a Mermaid flowchart in top-to-bottom (TB) orientation.
    """
    
    # Extract the key parts from the data
    reasoning_process = data["reasoning_process"]
    foundation_observations = reasoning_process["foundation_observations"]
    thoughts = reasoning_process["thoughts"]
    
    # We'll collect lines in a list, then join them at the end
    mermaid_lines = []
    mermaid_lines.append("flowchart TB\n")
    
    # We’ll store the created node IDs for atomic steps so we can link them properly
    # The key will be something like "FO_1_AS_1" or "TH_1_AS_21" 
    # and we'll store references to easily build connections after creation
    created_nodes = set()
    
    # ------------------------------------------------------
    # 1) Foundation Observations (FOs)
    # ------------------------------------------------------
    mermaid_lines.append("subgraph Foundation Observations")
    
    # For storing step connections: (node_from, node_to)
    fo_step_connections = []
    
    for fo in foundation_observations:
        fo_key = fo["key"]
        fo_name = fo["name"]
        mermaid_lines.append(f'    subgraph {fo_key}["{fo_key}: {fo_name}"]\n    direction TB')
        
        # Each FO has a list of atomic_steps
        for atom_step in fo["atomic_steps"]:
            as_key = atom_step["key"]
            # In the new model, previous_step_key may or may not exist.
            # If it doesn't exist in the dict, we default to "None".
            previous_step_key = atom_step.get("previous_step_key", "None")
            as_content = atom_step["content"]
            
            # Escape potential newlines
            as_content_escaped = as_content.replace("\n", "\\n")
            
            # Create a unique ID for this node
            node_id = f"{fo_key}_{as_key}"
            created_nodes.add(node_id)
            mermaid_lines.append(f'        {node_id}["{as_key}: {as_content_escaped}"]')
            
            # If there's a valid previous step, create a link from previous -> current
            if previous_step_key != "None":
                prev_node_id = f"{fo_key}_{previous_step_key}"
                fo_step_connections.append((prev_node_id, node_id))
        
        mermaid_lines.append("    end")  # End subgraph for this FO
    
    mermaid_lines.append("end\n")  # End "Foundation Observations" subgraph
    
    # ------------------------------------------------------
    # 2) Thoughts (THs)
    # ------------------------------------------------------
    mermaid_lines.append("subgraph Thoughts")
    
    # For storing step connections within thoughts
    th_step_connections = []
    
    for th in thoughts:
        th_key = th["key"]
        th_name = th["name"]
        th_backtracked = th["backtracked_from"]
        th_parent = th["parent_thought"]
        th_fo_associations = th["associated_foundation_observations"]
        guard_rails = th["guard_rails_to_consider"]
        thought_process = th["thought_process"]
        
        # Build a multiline label
        label_lines = [
            f"{th_key}: {th_name}",
            f"(backtracked_from: {th_backtracked})",
            f"(parent_thought: {th_parent})",
            f"Associated FOs: [{', '.join(th_fo_associations)}]"
        ]
        thought_label = "\\n".join(label_lines)
        
        # Start the subgraph for this Thought
        mermaid_lines.append(f'    subgraph {th_key}["{thought_label}"]')
        
        # --- Guard Rails ---
        mermaid_lines.append(f'        subgraph GR_{th_key}["Guard Rails"]')
        if len(guard_rails) == 0:
            mermaid_lines.append("            GR_None_1[No guard rails specified]")
        else:
            for i, gr in enumerate(guard_rails, start=1):
                if is_gpt_prompt:
                    gr_value = gr.replace("\n", "\\n")
                else:
                    gr_value = gr.name.replace("\n", "\\n")
                mermaid_lines.append(f'            GR_{th_key}_{i}["{gr_value}"]')
        mermaid_lines.append(f"        end")  # End of guard rails subgraph
        
        # --- Thought Process (Atomic Steps) ---
        mermaid_lines.append(f'        subgraph {th_key}_AtomicSteps["Thought Process"]\n    direction TB')
        for atom_step in thought_process:
            as_key = atom_step["key"]
            previous_step_key = atom_step.get("previous_step_key", "None")
            as_content = atom_step["content"]
            as_content_escaped = as_content.replace("\n", "\\n")
            
            # Unique ID for this step
            node_id = f"{th_key}_{as_key}"
            created_nodes.add(node_id)
            
            mermaid_lines.append(f'            {node_id}["{as_key}: {as_content_escaped}"]')
            
            # If there's a valid previous step, link them
            if previous_step_key != "None":
                prev_node_id = f"{th_key}_{previous_step_key}"
                th_step_connections.append((prev_node_id, node_id))
        
        mermaid_lines.append(f"        end")  # End subgraph of thought process
        
        mermaid_lines.append("    end")  # End subgraph for this Thought
    
    mermaid_lines.append("end\n")  # End "Thoughts" subgraph
    
    # ------------------------------------------------------
    # 3) Connect FO -> Thoughts (based on associated_foundation_observations)
    # ------------------------------------------------------
    for th in thoughts:
        th_key = th["key"]
        for fo_key in th["associated_foundation_observations"]:
            mermaid_lines.append(f"{fo_key} --> {th_key}")
    
    # ------------------------------------------------------
    # 4) Connect parent_thought -> child_thought
    # ------------------------------------------------------
    for th in thoughts:
        th_key = th["key"]
        parent_key = th["parent_thought"]
        if parent_key != "None":
            mermaid_lines.append(f"{parent_key} --> {th_key}")
    
    # ------------------------------------------------------
    # 5) Connect atomic steps within each FO
    # ------------------------------------------------------
    for (prev_node, current_node) in fo_step_connections:
        # Only connect if both nodes were created (to avoid missing reference)
        if prev_node in created_nodes and current_node in created_nodes:
            mermaid_lines.append(f"{prev_node} --> {current_node}")
    
    # ------------------------------------------------------
    # 6) Connect atomic steps within each TH
    # ------------------------------------------------------
    for (prev_node, current_node) in th_step_connections:
        if prev_node in created_nodes and current_node in created_nodes:
            mermaid_lines.append(f"{prev_node} --> {current_node}")
    
    # Join all lines
    return "\n".join(mermaid_lines)

# # Build the Mermaid diagram
# diagram_text = build_mermaid_diagram(response.model_dump())

# # Print or save the diagram text
# print(diagram_text)

In [16]:
def make_prompt_for_gpt_prompt(task: str, guardrails: List[GuardRail] = [], guardrail_enum: Enum = None):
    output = ""
    output += "<system prompt>\n"
    output += IN_DEPTH_THINKING_SYSTEM_PROMPT
    
    output += "\n\n # Response Model\n"
    output += "You must respond with a valid JSON object that matches the response model below:\n"
    output += "<response model>\n"
    output += json.dumps(openai_schema(get_classes_with_enum(guardrail_enum)).openai_schema, indent=2)
    output += "\n</response model>\n"
    output += "\n</system prompt>\n\n\n"
    output += "<user prompt>\n"
    output += format_structured_reasoning_user_prompt(task, guardrails)
    output += "\n</user prompt>\n"
    return output

In [17]:
print(make_prompt_for_gpt_prompt(TASK, GUARDRAILS, GuardRailEnum))

<system prompt>
# Purpose
You are an free thinking assistant that engages in extremely thorough, self-questioning reasoning. 
Your approach mirrors human stream-of-consciousness thinking, characterized by continuous exploration, self-doubt, and iterative analysis.

# Context
## Core Principles

### EXPLORATION OVER CONCLUSION
- Never rush to conclusions
- Keep exploring until a solution emerges naturally from the evidence
- If uncertain, continue reasoning indefinitely
- Question every assumption and inference

### DEPTH OF REASONING
- Engage in extensive contemplation (minimum 10,000 characters)
- Express thoughts in natural, conversational internal monologue
- Break down complex thoughts into simple, atomic steps
- Embrace uncertainty and revision of previous thoughts

### THINKING PROCESS
- Use short, simple sentences that mirror natural thought patterns
- Express uncertainty and internal debate freely
- Show work-in-progress thinking
- Acknowledge and explore dead ends
- Frequently

In [21]:
with open('../data/gpt-pro-response.json', 'r') as f:
    gpt_pro_response = json.load(f)
    
diagram_text = build_mermaid_diagram(gpt_pro_response['parameters'], is_gpt_prompt=True)
print(diagram_text)


flowchart TB

subgraph Foundation Observations
    subgraph FO_1["FO_1: Understanding Key Challenges in AI Era Education"]
    direction TB
        FO_1_AS_1["AS_1: We note that AI is changing what skills are most valuable."]
        FO_1_AS_2["AS_2: Critical thinking and creativity are increasingly important."]
        FO_1_AS_3["AS_3: Rapid technological shifts demand flexible learning pathways."]
        FO_1_AS_4["AS_4: Cultural sensitivity is also key for a global environment."]
        FO_1_AS_5["AS_5: Budget constraints remain a pragmatic factor."]
    end
    subgraph FO_2["FO_2: Balancing Creativity, Cultural Context, and Budget"]
    direction TB
        FO_2_AS_6["AS_6: We must ensure creativity remains a core outcome."]
        FO_2_AS_7["AS_7: We should consider ways to adapt curricula to diverse cultures."]
        FO_2_AS_8["AS_8: We can leverage AI to reduce certain costs and stay within budget."]
        FO_2_AS_9["AS_9: Programs that unify cultural differences can pre